In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file="worlds2025.xlsx"
df=pd.read_excel(file, sheet_name="Sheet1")

pick_cols=["Pick1","Pick2","Pick3","Pick4","Pick5"]
ban_cols=["Ban1","Ban2","Ban3","Ban4","Ban5"]

picks=df.melt(id_vars=["Match ID","Team","Win/Lose"],value_vars=pick_cols,
                var_name="PickSlot",value_name="Champion").dropna(subset=["Champion"])
picks["Champion"]=picks["Champion"].str.strip().str.title()
picks["Team"]=picks["Team"].str.strip()
picks["Win/Lose"]=picks["Win/Lose"].str.strip().str.title()

bans=df.melt(value_vars=ban_cols,value_name="Banned").dropna()
bans["Banned"]=bans["Banned"].str.strip().str.title()

match_count=df["Match ID"].nunique()

champ_stats=picks.groupby("Champion").agg(Played=("Champion", "count"),
    Wins=("Win/Lose",lambda x:(x=="Win").sum())
)
champ_stats["WinRate (%)"]=(champ_stats["Wins"]/champ_stats["Played"]*100).round(1)

ban_stats=bans["Banned"].value_counts().rename("Bans")

# Merge Pick & Ban Stats
summary=champ_stats.merge(ban_stats, left_index=True, right_index=True, how="outer").fillna(0)
summary["PickRate (%)"]=(summary["Played"]/(match_count*2)*100).round(1)
summary["BanRate (%)"]=(summary["Bans"]/(match_count*2)*100).round(1)

summary=summary[["Played","Wins","WinRate (%)","Bans","PickRate (%)","BanRate (%)"]]

# Top Picks & Bans
top_pick_val=summary["Played"].max()
top_ban_val=summary["Bans"].max()

top_picks=summary[summary["Played"]==top_pick_val].index.tolist()
top_bans=summary[summary["Bans"]==top_ban_val].index.tolist()

# Win Rate Analysis (>=5 games)
eligible=summary[summary["Played"]>= 5]
best_win=eligible[eligible["WinRate (%)"]==eligible["WinRate (%)"].max()].index.tolist() if not eligible.empty else []
worst_win=eligible[eligible["WinRate (%)"]==eligible["WinRate (%)"].min()].index.tolist() if not eligible.empty else []

# Unique Picks
team_unique=picks.groupby("Team")["Champion"].agg(lambda x: sorted(set(x))).reset_index()
team_unique["UniqueCount"] = team_unique["Champion"].apply(len)

print(f"Most Picked Champion(s): {top_picks} ({int(top_pick_val)} games)")
print(f"Most Banned Champion(s): {top_bans} ({int(top_ban_val)} bans)")
if best_win:
    print(f"Best Win Rate (min 5 games): {best_win}")
if worst_win:
    print(f"Worst Win Rate (min 5 games): {worst_win}")

print("\nUnique Champions per Team:")
for _, row in team_unique.sort_values("UniqueCount", ascending=False).iterrows():
    print(f"{row['Team']}: {row['UniqueCount']} champions")

top10_picks = summary.sort_values("PickRate (%)", ascending=False).head(10)
top10_bans = summary.sort_values("BanRate (%)", ascending=False).head(10)

plt.figure(figsize=(9, 5))
plt.bar(top10_picks.index,top10_picks["PickRate (%)"],color="skyblue")
plt.title("Top 10 Champions by Pick Rate")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(9, 5))
plt.bar(top10_bans.index,top10_bans["BanRate (%)"],color="salmon")
plt.title("Top 10 Champions by Ban Rate")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
